In [13]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
dataType = {'IsPublishedForNeg':object,'IsFromInternet':object,'IsFromVista':object,'TerminalID':object,'InternetOffice':object}
%time sb = pd.read_csv('res.csv', header=0, sep=',', dtype=dataType,
                usecols=['OfficeID','Country','Origin','Destination','RoundTrip','Seg1Date','Seg2Date',
                        'From','IsPublishedForNeg','IsFromInternet','IsFromVista','TerminalID','InternetOffice','IsBooked'])

CPU times: user 2min 3s, sys: 1min 16s, total: 3min 19s
Wall time: 5min 41s


In [3]:
len(sb.index)

20390198

In [4]:
sb.head(3)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg2Date,Seg2Carrier,Seg2BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,IsBooked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,2013-02-02,D2,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,2013-08-09,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0


In [5]:
booked = sb.loc[sb['IsBooked'] == 1]

In [6]:
len(booked.index)

1098257

In [23]:
# Ensure that data is in the correct data type
def prepare_df(df, int_cols, date_cols):
    nonnumerics = set([k for k in df.keys() if k not in floats+dates])
    # for c in ints:
    #    df[c] = df[c].astype(int)
    for c in floats:
        df[c] = df[c].astype(float)
    for c in dates:
        df[c] = pd.to_datetime(df[c], errors='coerce')
    for c in nonnumerics:
        df[c] = df[c].astype(str)
    #encode the labels for the categorical features
    for f in nonnumerics:
        le = LabelEncoder()
        df[f] = le.fit_transform(df[f].astype(str))
    return df

# ints = ['NbSegments']
floats = ['NbSegments']
dates = ['Date', 'Seg1Date', 'Seg2Date']

In [24]:
%time data = prepare_df(sb, ints, dates)

CPU times: user 6min 53s, sys: 6min 38s, total: 13min 32s
Wall time: 16min 25s


In [26]:
# Separat into input and label data
labels = data['IsBooked']
features = data.drop('IsBooked', axis=1)

In [8]:
#create training and test datasets
%time train_X, test_X, train_Y, test_Y = train_test_split(features, labels)

CPU times: user 1min 36s, sys: 42.8 s, total: 2min 19s
Wall time: 2min 55s


In [9]:
gbm_s = xgb.XGBClassifier(max_depth=30, n_estimators=750, learning_rate=0.05)

In [10]:
%time train_X.as_matrix().shape

(15292648, 25)

In [11]:
%time test_X.as_matrix().shape

CPU times: user 11.9 s, sys: 8.41 s, total: 20.3 s
Wall time: 24.3 s


(5097550, 25)

In [12]:
%time gbm_s_fit = gbm_s.fit(train_X.as_matrix(), train_Y.as_matrix(), eval_metric="auc")

ValueError: could not convert string to float: '2013-10-14'

In [ ]:
%time pred = gbm_s.predict(test_X.as_matrix())

In [ ]:
metrics.accuracy_score(test_Y, pred)

In [ ]:
metrics.confusion_matrix(test_Y, pred)

In [ ]:
def my_plot_importance(booster, figsize, **kwargs):
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return xgb.plot_importance(booster=booster, ax=ax, **kwargs)

def my_plot_tree(booster, figsize, **kwargs): 
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return xgb.plot_tree(booster=booster, ax=ax, **kwargs)

In [ ]:
my_plot_importance(gbm_s, (15,15))

In [ ]:
pred_s_prob = gbm_s.predict_proba(test_X.as_matrix())

pred_s_T_prob = pred_s_prob.transpose()[0].transpose()
pred_s_F_prob = pred_s_prob.transpose()[1].transpose()

fpr, tpr, thr = metrics.roc_curve(test_Y, pred_s_F_prob)
roc_auc = metrics.auc(fpr, tpr)